# Graduierte Algebra der gemischten Differentialformen

Sei $M,N$ jeweils eine differenzierbare $m$- bzw. $n$-dimensionale Mannigfaltigkeit. Ferner sei $\varphi: M \to N$ differenzierbar. Der Raum der $k$-Formen $\Omega^k(M,\varphi)=\Gamma \left( \bigwedge^k \left( \varphi^* T^*N \right) \right)$ auf $M$ entlang $\varphi$ ist ein Modul über $C^\infty(M)$. Durch das Wedge-Produkt erhalten wir eine Multiplikation zwischen diesen Formen und gewinnen damit eine graduierte Algebra $\Omega^*(M, \varphi)=\bigoplus^n_{k=0} \Omega^k(M, \varphi)$ über wahlweise $\mathbb{R}$ oder $\mathbb{C}$, die wir nun implementieren wollen. Ziel ist es, später Matrix-Operationen im Sage-Framework über dieser Algebra durchzuführen.

Jetzt probieren wir mal ein wenig herum und definieren uns dafür zunächst eine Mannigfaltigkeit mit Karte

In [1]:
M = Manifold(2, 'M')
X.<x,y> = M.chart()

und die entsprechenden Räume:

In [2]:
Omega0 = M.diff_form_module(0); print(Omega0)
Omega1 = M.diff_form_module(1); print(Omega1)
Omega2 = M.diff_form_module(2); print(Omega2)

Algebra of differentiable scalar fields on the 2-dimensional differentiable manifold M
Free module Omega^1(M) of 1-forms on the 2-dimensional differentiable manifold M
Free module Omega^2(M) of 2-forms on the 2-dimensional differentiable manifold M


In [3]:
Omega = M.mixed_form_algebra(); print(Omega)

Graded algebra Omega^*(M) of mixed differential forms on the 2-dimensional differentiable manifold M


In [4]:
print(Omega.category())

Category of graded algebras over Symbolic Ring


In [5]:
print(Omega.base_ring())

Symbolic Ring


Nun definieren wir ein paar Formen:

In [6]:
f = M.scalar_field(x, name='f'); print(f)
a1 = M.diff_form(1, name='a_1'); print(a1)
a2 = M.diff_form(1, name='a_2'); print(a2)
b = M.diff_form(2, name='b'); print(b)

Scalar field f on the 2-dimensional differentiable manifold M
1-form a_1 on the 2-dimensional differentiable manifold M
1-form a_2 on the 2-dimensional differentiable manifold M
2-form b on the 2-dimensional differentiable manifold M


und ordnen ihnen noch ein paar Werte zu:

In [7]:
show(f.display())
a1[X.frame(),0] = y*x^2; show(a1.display())
a2[X.frame(),1] = y; show(a2.display())
b[X.frame(),0,1] = y^2*x; show(b.display())

f: M --> R
   (x, y) |--> x

a_1 = x^2*y dx

a_2 = y dy

b = x*y^2 dx/\dy

In [8]:
A = Omega([f,a1,b], name='A'); print(A)

Mixed differential form A on the 2-dimensional differentiable manifold M


In [9]:
print(A.display())
show(A.display())

A = f + a_1 + b


A = f + a_1 + b

Für die Ausgabe in Koordinaten müssen wir lediglich einen Rahmen angeben:

In [10]:
show(A.disp(X.frame()))

A = [x] + [x^2*y dx] + [x*y^2 dx/\dy]

Schauen wir einmal, ob wir die gemischten Differentialformen und deren Arithmetik richtig implementiert haben:

In [11]:
B = A + a2; show(B.disp(X.frame()))
C = A - A*B; show(C.disp(X.frame())) # Die disp-Ausgabe darf durchaus noch verbessert werden...
D = A - B*A; show(D.disp(X.frame())) # Die Multiplikation ist offensichtlich nicht kommutativ!

A+a_2 = [x] + [x^2*y dx + y dy] + [x*y^2 dx/\dy]

A-A/\(A+a_2) = [-x^2 + x] + [-(2*x^3 - x^2)*y dx - x*y dy] + [-(3*x^2 - x)*y^2 dx/\dy]

A-(A+a_2)/\A = [-x^2 + x] + [-(2*x^3 - x^2)*y dx - x*y dy] + [-(x^2 - x)*y^2 dx/\dy]

Sieht gut aus. Zur Überprüfung schauen wir uns einmal die Coercions an:

In [12]:
show(Omega.coerce(f).display())
show(Omega.coerce(a2).display())
show((Omega.coerce(f)*Omega.coerce(a2)).display())
show((Omega.coerce(a2)*Omega.coerce(a1)).display())

f = f + zero + zero

a_2 = zero + a_2 + zero

f/\a_2 = zero + (unnamed 1-form) + (unnamed 2-form)

a_2/\a_1 = zero + (unnamed 1-form) + (unnamed 2-form)

Wunderbar! Die Coercion scheint vollständig implementiert. Können wir nun also auch sagen, ob die entsprechenden Differentialformen in $\Omega^*(M)$ leben? Ja, das können wir!

In [13]:
print(a1 in Omega1)
print(a1 in Omega)
print(f in Omega0)
print(f in Omega)

True
True
True
True


Komponentenmanipulation der Klasse:

In [14]:
show(A[0].display())
show(A[2].display())

f: M --> R
   (x, y) |--> x

b = x*y^2 dx/\dy

In [15]:
show(A.display())
A[1]=a2; show(A.display())

A = f + a_1 + b

A = f + a_2 + b

In [16]:
A[:] = [f,0,0]; print(A[:])

[Scalar field f on the 2-dimensional differentiable manifold M, 1-form zero on the 2-dimensional differentiable manifold M, 2-form zero on the 2-dimensional differentiable manifold M]


Versuchen wir nun ein wenig mit Matrizen rumzurechnen:

In [17]:
MS = MatrixSpace(Omega, 2); print(MS)

Full MatrixSpace of 2 by 2 dense matrices over Graded algebra Omega^*(M) of mixed differential forms on the 2-dimensional differentiable manifold M


In [18]:
m1 = MS.matrix([a1,Omega.zero(),Omega.zero(),a2]); show(m1) # Jaja, die Ausgabe muss verbessert werden...
m2 = MS.matrix([a2,Omega.zero(),Omega.zero(),a1]); show(m2)

[ Mixed differential form a_1 on the 2-dimensional differentiable manifold M Mixed differential form zero on the 2-dimensional differentiable manifold M]
[Mixed differential form zero on the 2-dimensional differentiable manifold M  Mixed differential form a_2 on the 2-dimensional differentiable manifold M]

[ Mixed differential form a_2 on the 2-dimensional differentiable manifold M Mixed differential form zero on the 2-dimensional differentiable manifold M]
[Mixed differential form zero on the 2-dimensional differentiable manifold M  Mixed differential form a_1 on the 2-dimensional differentiable manifold M]

In [19]:
show(m1.determinant().disp(X.frame())) # Okay, das Ergebnis stimmt schonmal. Und...
show(m2.determinant().disp(X.frame())) # ...glücklicherweise beachtet Sage beim Determinieren auch die Reihenfolge!

a_1/\a_2-zero/\zero = [0] + [0] + [x^2*y^2 dx/\dy]

a_2/\a_1-zero/\zero = [0] + [0] + [-x^2*y^2 dx/\dy]

Wie ist denn das jetzt mit den Restrictions? Probieren wir wieder mal ein wenig herum:

In [20]:
U = M.open_subset('U')
print(A.restrict(U))
print(B.restrict(U))

Mixed differential form A on the Open subset U of the 2-dimensional differentiable manifold M
Mixed differential form A+a_2 on the Open subset U of the 2-dimensional differentiable manifold M


In [21]:
OmegaU = U.mixed_form_algebra(); print(OmegaU)

Graded algebra Omega^*(U) of mixed differential forms on the Open subset U of the 2-dimensional differentiable manifold M


In [22]:
print(OmegaU.coerce(A))

Mixed differential form A on the Open subset U of the 2-dimensional differentiable manifold M


Okay, momentan war die Mannigfaltigkeit noch parallelisierbar. Was ist aber, wenn nicht?

In [23]:
S2 = Manifold(2, name='S2', latex_name=r'S^2', start_index=1); print(S2)

2-dimensional differentiable manifold S2


In [24]:
U = S2.open_subset(name='U', latex_name=r'S^2 \setminus \{\text{South pole}\}'); show(U)
V = S2.open_subset(name='V', latex_name=r'S^2 \setminus \{\text{North pole}\}'); show(V)
S2.declare_union(U,V)

Open subset U of the 2-dimensional differentiable manifold S2

Open subset V of the 2-dimensional differentiable manifold S2

In [25]:
c_tz.<t,z> = U.chart(); print(c_tz)
c_uv.<u,v> = V.chart(); print(c_uv)

Chart (U, (t, z))
Chart (V, (u, v))


In [26]:
tz_to_uv = c_tz.transition_map(c_uv, (t/(t^2+z^2), z/(t^2+z^2)),
                               intersection_name='W', restrictions1= t^2+z^2!=0,
                               restrictions2= u^2+v^2!=0); print(tz_to_uv)
uv_to_tz = tz_to_uv.inverse(); print(uv_to_tz)

Change of coordinates from Chart (W, (t, z)) to Chart (W, (u, v))
Change of coordinates from Chart (W, (u, v)) to Chart (W, (t, z))


In [27]:
e_tz = c_tz.frame(); print(e_tz)
e_uv = c_uv.frame(); print(e_uv)

Coordinate frame (U, (d/dt,d/dz))
Coordinate frame (V, (d/du,d/dv))


In [28]:
g = S2.scalar_field(t, name='g'); print(g)
omega = S2.diff_form(1, name='omega', latex_name=r'\omega'); print(omega)
eta = S2.diff_form(1, name='eta', latex_name=r'\eta'); print(eta)

Scalar field g on the 2-dimensional differentiable manifold S2
1-form omega on the 2-dimensional differentiable manifold S2
1-form eta on the 2-dimensional differentiable manifold S2


In [29]:
omega[e_tz,:] = -t^2, z^2; show(omega.disp(e_tz))
eta[e_uv,:] = v^2, -u; show(eta.disp(e_uv))

omega = -t^2 dt + z^2 dz

eta = v^2 du - u dv

In [30]:
OmegaS2 = S2.mixed_form_algebra(); print(OmegaS2)

Graded algebra Omega^*(S2) of mixed differential forms on the 2-dimensional differentiable manifold S2


In [31]:
F = OmegaS2([g,omega,0], name='F'); print(F); show(F.disp(e_tz))
G = OmegaS2([0,eta,0], name='G'); print(G); show(G.disp(e_uv))

Mixed differential form F on the 2-dimensional differentiable manifold S2


F = [t] + [-t^2 dt + z^2 dz] + [0]

Mixed differential form G on the 2-dimensional differentiable manifold S2


G = [0] + [v^2 du - u dv] + [0]

Damit wir auch in allen Koordinaten mit den gemischten Formen rechnen können, müssen wir Sage anweisen, die entsprechenden Trafos durchzuführen. Glücklicherweise können wir dank unserer Implementierung alle Differentialformen mit einer Fliege schlagen:

In [32]:
F.add_comp_by_continuation(e_uv, V.intersection(U), c_uv); show(F.disp(e_uv))
G.add_comp_by_continuation(e_tz, U.intersection(V), c_tz); show(G.disp(e_tz))

F = [u/(u^2 + v^2)] + [(u^4 - u^2*v^2 - 2*u*v^3)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) du + (2*u^3*v + u^2*v^2 - v^4)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) dv] + [0]

G = [0] + [(2*t^4*z + 2*t^2*z^3 - t^2*z^2 + z^4)/(t^8 + 4*t^6*z^2 + 6*t^4*z^4 + 4*t^2*z^6 + z^8) dt - (t^5 - t*z^4 + 2*t*z^3)/(t^8 + 4*t^6*z^2 + 6*t^4*z^4 + 4*t^2*z^6 + z^8) dz] + [0]

In [33]:
show((F*G).disp(e_uv))
show((F*G).disp(e_tz))

F/\G = [0] + [u*v^2/(u^2 + v^2) du - u^2/(u^2 + v^2) dv] + [-(u^2*v^4 - v^6 + u^5 - u^3*v^2 + 2*(u^3 - u^2)*v^3)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) du/\dv]

F/\G = [0] + [(2*t^5*z + 2*t^3*z^3 - t^3*z^2 + t*z^4)/(t^8 + 4*t^6*z^2 + 6*t^4*z^4 + 4*t^2*z^6 + z^8) dt - (t^6 - t^2*z^4 + 2*t^2*z^3)/(t^8 + 4*t^6*z^2 + 6*t^4*z^4 + 4*t^2*z^6 + z^8) dz] + [(t^7 - 2*t^2*z^5 - z^6 - (t^3 - t^2)*z^4 - 2*(t^4 - t^3)*z^3)/(t^8 + 4*t^6*z^2 + 6*t^4*z^4 + 4*t^2*z^6 + z^8) dt/\dz]

In [34]:
show((F*G).exterior_derivative().disp(e_uv))

d(F/\G) = [0] + [0] + [-2*(u^3*v + u*v^2)/(u^4 + 2*u^2*v^2 + v^4) du/\dv]